In [35]:
import numpy as np

In [ ]:
class VRP:
    def __init__(
        self,
        population_size,
        dimensions,
        bounds,
        Mutation_rate,
        Crossover_rate,
        distance,
    ):
        # Input params
        self.population_size = population_size
        self.dimensions = dimensions
        self.bounds = bounds
        self.Mutation_rate = Mutation_rate
        self.Crossover_rate = Crossover_rate

        self.distance = distance

        # Internal params
        self.global_solution = np.array([])
        self.F = Mutation_rate[0]
        self.CR = Crossover_rate[0]
        self.current_cost = np.array([])
        self.kwargs = {"distance": self.distance}

        # Derived internal
        self.population = None
        self.Upperbound_Mutation = None
        self.Lowerbound_Mutation = None
        self.Upperbound_Crossover_rate = None
        self.Lowerbound_Crossover_rate = None

    def reset(self):
        # Initialize population
        self.population = np.random.uniform(
            self.bounds[:, 0],
            self.bounds[:, 1],
            (self.population_size, len(self.bounds)),
        )
        self.Upperbound_Mutation = self.Mutation_rate[1]
        self.Lowerbound_Mutation = self.Mutation_rate[0]
        self.Upperbound_Crossover_rate = self.Crossover_rate[1]
        self.Lowerbound_Crossover_rate = self.Crossover_rate[0]
        self.F = self.Mutation_rate[0]
        self.CR = self.Crossover_rate[0]

    def preserving_strategy(self, X, **kwargs):
        # distance matrix
        distance = kwargs["distance"]
        # total distance starts from zero km.
        total_distance = 0
        # Vehicle travel from depot to customer i
        total_distance += distance[0][X[0]]
        # Total distance of routing solution
        for i in range(len(X) - 1):
            total_distance += distance[X[i]][X[i + 1]]
        # Vehicle returns to depot
        total_distance += distance[X[-1]][0]
        # Return total distance (km.) that vehicle traveled
        return total_distance

    def f_per_particle(self, m, **kwargs):
        X = m  # Sequence
        obj_val = self.preserving_strategy(X, **kwargs)  # Call Preserving strategy.
        return obj_val

    def objective_func(self, x, **kwargs):
        """Decoding of each particles for obtaining routing solutions by argsort()"""
        seq = x.argsort() + 1
        """Calculate objective function for obtaining objective value of each particle"""
        j = self.f_per_particle(seq, **kwargs)
        return np.array(j)

    def evolve(self, n_iteration):
        
        Upperbound_Mutation = self.Upperbound_Mutation
        Lowerbound_Mutation = self.Lowerbound_Mutation
        Upperbound_Crossover_rate = self.Upperbound_Crossover_rate
        Lowerbound_Crossover_rate = self.Lowerbound_Crossover_rate
        population_size = self.population_size
        bounds = self.bounds
        max_generations = n_iteration

        for _ in range(max_generations):
            # print(f'Iteration {generation}')
            current_cost = np.array([])
            self.F += (Upperbound_Mutation - Lowerbound_Mutation) / max_generations
            self.CR += (
                Upperbound_Crossover_rate - Lowerbound_Crossover_rate
            ) / max_generations
            for i in range(population_size):
                # Mutation
                indices = [idx for idx in range(population_size) if idx != i]
                a, b, c = self.population[np.random.choice(indices, 3, replace=False)]
                mutant = self.population[i] + self.F * (b - c)

                # Crossover
                crossover_prob = np.random.rand(len(bounds))
                trial = np.where(crossover_prob < self.CR, mutant, self.population[i])

                # Selection
                fitness_trial = self.objective_func(trial, **self.kwargs)
                fitness_current = self.objective_func(self.population[i], **self.kwargs)

                if fitness_trial < fitness_current:
                    self.population[i] = trial
                    current_cost = np.insert(
                        current_cost, len(current_cost), fitness_trial
                    )
                else:
                    current_cost = np.insert(
                        current_cost, len(current_cost), fitness_current
                    )
                print(f"population {i}")
                print(f"current_cost{current_cost}")
                print("---------" * 30)
            
            
            best_index_plot = current_cost[np.argmin(current_cost)]
            self.global_solution = np.insert(
                self.global_solution, len(self.global_solution), best_index_plot
            )
        # Find the best solution
        best_index = np.argmin(
            [
                self.objective_func(individual, **self.kwargs)
                for individual in self.population
            ]
        )
        best_solution = self.population[best_index]

        return best_solution, self.global_solution
    
    
    
    def get_best_solution(self):
        # Evaluate the objective value for each individual in the population.
        obj_values = [self.objective_func(ind, **self.kwargs) for ind in self.population]

        # Find the index of the individual with the lowest (best) objective value.
        best_index = np.argmin(obj_values)

        # Get the best objective value and the corresponding individual (solution).
        best_solution = obj_values[best_index]
        best_position = self.population[best_index]
        
        #return best_solution, best_position
        return best_solution

        

In [37]:
distance = np.array(
    [
        [0, 0.664, 1.035, 1.789, 4.854, 7.586, 11.425, 11.871],
        [4.857, 0, 0.852, 1.606, 4.671, 7.403, 11.242, 11.688],
        [4.004, 4.23, 0, 0.753, 3.818, 6.55, 10.389, 10.835],
        [5.857, 6.083, 6.454, 0, 3.064, 5.796, 9.635, 10.081],
        [7.267, 7.493, 7.864, 4.066, 0, 2.934, 6.698, 7.144],
        [10.704, 10.93, 11.301, 7.503, 3.68, 0, 5.619, 6.065],
        [13.475, 13.701, 14.072, 10.274, 6.451, 5.409, 0, 0.943],
        [15.079, 15.305, 15.676, 11.878, 8.055, 7.013, 1.603, 0],
    ]
)


dimensions = len(distance) - 1
maxiters = 100
population_size = 100
bounds = np.array([[0, 1]] * dimensions)
Mutation_rate = np.array([0.9, 0.5])
Crossover_rate = np.array([0.5, 0.1])


vrp = VRP(
    population_size=population_size,
    dimensions=dimensions,
    bounds=bounds,
    Mutation_rate=Mutation_rate,
    Crossover_rate=Crossover_rate,
    distance=distance,
)

In [38]:
vrp.reset()
best_solution, global_solution_plot = vrp.evolve(n_iteration=10)

population 0
current_cost[53.28]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
population 1
current_cost[53.28  72.527]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
population 2
current_cost[53.28  72.527 72.527]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
population 3
current_cost[53.28  72.527 72.527 50.14 ]
----------

In [39]:
vrp.get_best_solution()

(array(29.949),
 array([0.08863718, 0.56307356, 0.58104547, 0.84921324, 0.65573863,
        0.70867551, 0.81592687]))

In [40]:
routing = best_solution.argsort() + 1
routing = np.insert(routing, len(routing), 0)
routing = np.insert(routing, 0, 0)
print(f" Routing: {routing}")

 Routing: [0 1 2 3 5 6 7 4 0]


In [41]:
# print(f' Total distance {preserving_strategy(routing, **kwargs)} KM.')

In [42]:
vrp.reset()
vrp.get_best_solution()

(array(36.619),
 array([0.67959783, 0.17743066, 0.90145784, 0.18012687, 0.47863485,
        0.64984385, 0.59029443]))